In [1]:
import os, sys
# add to path
sys.path.append("../..")

import attr
import datetime

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import seaborn

import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_probability as tfp

import pandas as pd
import sonnet as snt

from filterflow.smc import SMC
from filterflow.base import State, StateWithMemory, StateSeries, DTYPE_TO_OBSERVATION_SERIES, DTYPE_TO_STATE_SERIES

from filterflow.observation.base import ObservationModelBase, ObservationSampler
from filterflow.observation.linear import LinearObservationSampler
from filterflow.transition.random_walk import RandomWalkModel
from filterflow.proposal import BootstrapProposalModel
from filterflow.transition.base import TransitionModelBase

from filterflow.resampling.criterion import NeffCriterion, AlwaysResample, NeverResample
from filterflow.resampling.standard import SystematicResampler, MultinomialResampler
from filterflow.resampling.differentiable import RegularisedTransform, CorrectedRegularizedTransform
from filterflow.resampling.differentiable.ricatti.solver import RicattiSolver

from filterflow.resampling.base import NoResampling

from filterflow.state_space_model import StateSpaceModel

## Load Data

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import pickle

import numpy as np
from scipy.sparse import coo_matrix
data_dir = "../../data/piano_data"

def sparse_pianoroll_to_dense(pianoroll, min_note, num_notes):
    """Converts a sparse pianoroll to a dense numpy array.
    Given a sparse pianoroll, converts it to a dense numpy array of shape
    [num_timesteps, num_notes] where entry i,j is 1.0 if note j is active on
    timestep i and 0.0 otherwise.
    Args:
    pianoroll: A sparse pianoroll object, a list of tuples where the i'th tuple
      contains the indices of the notes active at timestep i.
    min_note: The minimum note in the pianoroll, subtracted from all notes so
      that the minimum note becomes 0.
    num_notes: The number of possible different note indices, determines the
      second dimension of the resulting dense array.
    Returns:
    dense_pianoroll: A [num_timesteps, num_notes] numpy array of floats.
    num_timesteps: A python int, the number of timesteps in the pianoroll.
    """
    num_timesteps = len(pianoroll)
    inds = []
    for time, chord in enumerate(pianoroll):
        # Re-index the notes to start from min_note.
        inds.extend((time, note-min_note) for note in chord)
        shape = [num_timesteps, num_notes]
    values = [1.] * len(inds)
    sparse_pianoroll = coo_matrix(
      (values, ([x[0] for x in inds], [x[1] for x in inds])),
      shape=shape)
    return sparse_pianoroll.toarray(), num_timesteps

def create_pianoroll_dataset(path,
                             split,
                             batch_size,
                             num_parallel_calls=4,
                             shuffle=False,
                             repeat=False,
                             min_note=21,
                             max_note=108):
    """Creates a pianoroll dataset.
    Args:
    path: The path of a pickle file containing the dataset to load.
    split: The split to use, can be train, test, or valid.
    batch_size: The batch size. If repeat is False then it is not guaranteed
      that the true batch size will match for all batches since batch_size
      may not necessarily evenly divide the number of elements.
    num_parallel_calls: The number of threads to use for parallel processing of
      the data.
    shuffle: If true, shuffles the order of the dataset.
    repeat: If true, repeats the dataset endlessly.
    min_note: The minimum note number of the dataset. For all pianoroll datasets
      the minimum note is number 21, and changing this affects the dimension of
      the data. This is useful mostly for testing.
    max_note: The maximum note number of the dataset. For all pianoroll datasets
      the maximum note is number 108, and changing this affects the dimension of
      the data. This is useful mostly for testing.
    Returns:
    inputs: A batch of input sequences represented as a dense Tensor of shape
      [time, batch_size, data_dimension]. The sequences in inputs are the
      sequences in targets shifted one timestep into the future, padded with
      zeros. This tensor is mean-centered, with the mean taken from the pickle
      file key 'train_mean'.
    targets: A batch of target sequences represented as a dense Tensor of
      shape [time, batch_size, data_dimension].
    lens: An int Tensor of shape [batch_size] representing the lengths of each
      sequence in the batch.
    mean: A float Tensor of shape [data_dimension] containing the mean loaded
      from the pickle file.
    """
    # Load the data from disk.
    num_notes = max_note - min_note + 1
    with tf.io.gfile.GFile(path, "rb") as f:
        raw_data = pickle.load(f)
    pianorolls = raw_data[split]
    mean = raw_data["train_mean"]
    num_examples = len(pianorolls)

    def pianoroll_generator():
        for sparse_pianoroll in pianorolls:
            yield sparse_pianoroll_to_dense(sparse_pianoroll, min_note, num_notes)

    dataset = tf.data.Dataset.from_generator(
      pianoroll_generator,
      output_types=(tf.float64, tf.int64),
      output_shapes=([None, num_notes], []))

    if repeat: 
        dataset = dataset.repeat()
    if shuffle: 
        dataset = dataset.shuffle(num_examples)

    # Batch sequences togther, padding them to a common length in time.
    dataset = dataset.padded_batch(batch_size,
                                 padded_shapes=([None, num_notes], []))

    def process_pianoroll_batch(data, lengths):
        """Create mean-centered and time-major next-step prediction Tensors."""
        data = tf.cast(tf.transpose(data, perm=[1, 0, 2]), float)
        lengths = tf.cast(lengths, tf.int32)
        targets = data
        # Mean center the inputs.
        inputs = data - tf.constant(mean, dtype=tf.float32,
                                    shape=[1, 1, mean.shape[0]])
        # Shift the inputs one step forward in time. Also remove the last timestep
        # so that targets and inputs are the same length.
        inputs = tf.pad(inputs, [[1, 0], [0, 0], [0, 0]], mode="CONSTANT")[:-1]
        # Mask out unused timesteps.
        inputs *= tf.expand_dims(tf.transpose(
            tf.sequence_mask(lengths, dtype=inputs.dtype)), 2)
        return inputs, targets, lengths

    dataset = dataset.map(process_pianoroll_batch,
                        num_parallel_calls=num_parallel_calls)
    dataset = dataset.prefetch(num_examples)

    itr = tf.compat.v1.data.make_one_shot_iterator(dataset)
    inputs, targets, lengths = itr.get_next()
    return inputs, targets, lengths, tf.constant(mean, dtype=tf.float32)

path = os.path.join(data_dir, 'jsb.pkl')
inputs, targets, lens, mean = create_pianoroll_dataset(path, split='train', batch_size=1)

In [4]:


T = targets.shape.as_list()[0]
observation_size = targets.shape.as_list()[-1]


latent_size = 10
fcnet_hidden_sizes = [latent_size]
encoded_data_size = latent_size
rnn_hidden_size = latent_size//2

latent_encoder_layers = [32]
latent_encoded_size = 32

latent_encoder = snt.nets.MLP(
            output_sizes=latent_encoder_layers + [latent_encoded_size],
            name="latent_encoder")

data_encoder_layers = [32]
encoded_data_size = 32
data_encoder = snt.nets.MLP(
            output_sizes=data_encoder_layers + [encoded_data_size],
            name="data_encoder")

## Preamble

In [5]:
# store observations
batch_size = 1
n_particles = 100
dimension = latent_size

In [6]:
inputs = tf.tile(inputs, [1, batch_size*n_particles, 1])
targets = tf.tile(targets, [1, batch_size*n_particles, 1])

obs_data = tf.data.Dataset.from_tensor_slices(targets)
inputs_data = tf.data.Dataset.from_tensor_slices(inputs)

In [7]:
class NNNormalDistribution(tf.Module):
    """A Normal distribution with mean and var parametrised by NN"""

    def __init__(self, 
                 size, 
                 hidden_layer_sizes, 
                 sigma_min=0.0,
                 raw_sigma_bias=0.25, 
                 hidden_activation_fn=tf.nn.relu,
                 name="conditional_normal_distribution"):
        
        super(NNNormalDistribution, self).__init__(name=name)
        
        self.sigma_min = sigma_min
        self.raw_sigma_bias = raw_sigma_bias
        self.size = size
        self.fcnet = snt.nets.MLP(
            output_sizes=hidden_layer_sizes + [2*size],
            activation=hidden_activation_fn,
            activate_final=False,
            name=name + "_fcnet")

    def get_params(self, tensor_list, **unused_kwargs):
        """Computes the parameters of a normal distribution based on the inputs."""
        inputs = tf.concat(tensor_list, axis=-1)
        outs = self.fcnet(inputs)
        mu, sigma = tf.split(outs, 2, axis=-1)
        sigma = tf.maximum(tf.nn.softplus(sigma + self.raw_sigma_bias), self.sigma_min)
        return mu, sigma

    def __call__(self, *args, **kwargs):
        """Creates a normal distribution conditioned on the inputs."""
        mu, sigma = self.get_params(args, **kwargs)
        return tfp.distributions.Normal(loc=mu, scale=sigma)

## Transition

In [8]:
class VRNNTransitionModel(TransitionModelBase):
    def __init__(self, 
                 rnn_hidden_size, 
                 data_encoder, 
                 latent_encoder,
                 name='NNTransitionModel'):
        
        super(VRNNTransitionModel, self).__init__(name=name)
        
        # mlp parametrised gaussian
        self.transition = NNNormalDistribution(size=latent_size, 
                                               hidden_layer_sizes=[latent_size])
        # encoder for inputs
        self.latent_encoder = latent_encoder
        
        # lstm cell
        self.rnn_hidden_size = rnn_hidden_size
        self.lstm_cell = tf.keras.layers.LSTMCell(units=rnn_hidden_size)
        
    def run_rnn(self, state: State, inputs: tf.Tensor):
        
        # process latent state
        latent_state = state.particles
        
        # encode and reshape latent state
        latent_encoded = self.latent_encoder(latent_state)
        
        B,N, D = latent_encoded.shape
        latent_encoded_reshaped = tf.reshape(latent_encoded, [B*N,D])
        
        # process rnn_state
        rnn_state = tf.reshape(state.rnn_state, [B*N,rnn_hidden_size*2])
        rnn_state = tf.split(rnn_state, 2, axis=-1)
        
        
        # run rnn
        rnn_inputs = tf.concat([inputs, latent_encoded_reshaped], axis=-1)
        rnn_out, rnn_state = self.lstm_cell(rnn_inputs, rnn_state)
        
        rnn_state = tf.concat(rnn_state, axis=-1)
        rnn_state = tf.reshape(rnn_state, [state.batch_size, state.n_particles, rnn_hidden_size*2])
        return rnn_out, rnn_state, latent_encoded
    
    def latent_dist(self, rnn_out):
        rnn_out = tf.reshape(rnn_out, [state.batch_size, state.n_particles, rnn_hidden_size])
        dist = self.transition(rnn_out)
        return dist

    def loglikelihood(self, prior_state: State, proposed_state: State, inputs: tf.Tensor):
        rnn_out, rnn_state, latent_encoded = self.run_rnn(state, inputs)
        dist = self.latent_dist(rnn_out)
        new_latent = proposed_state.particles
        return tf.reduce_sum(dist.log_prob(new_latent), axis=-1)

    def sample(self, state: State, inputs: tf.Tensor):
        
        rnn_out, rnn_state, latent_encoded = self.run_rnn(state, inputs)
        dist = self.latent_dist(rnn_out)
        latent_state = dist.sample()
        
        return VRNNState(particles=latent_state, 
                          log_weights = state.log_weights,
                          weights=state.weights, 
                          log_likelihoods=state.log_likelihoods,
                          rnn_state = rnn_state,
                          rnn_out = rnn_out,
                          latent_encoded =  latent_encoded)

In [9]:
class VRNNProposalModel(VRNNTransitionModel):
    def __init__(self, 
                 rnn_hidden_size, 
                 data_encoder, 
                 latent_encoder,
                 name='VRNNProposalModel'):
        
        super(VRNNProposalModel, self).__init__(rnn_hidden_size, 
                 data_encoder, 
                 latent_encoder,name)

    def loglikelihood(self, proposed_state: State, state: State, inputs: tf.Tensor, observation: tf.Tensor):
        rnn_out, rnn_state, latent_encoded = self.run_rnn(state, inputs)
        dist = self.latent_dist(rnn_out)
        new_latent = proposed_state.particles
        return tf.reduce_sum(dist.log_prob(new_latent), axis=-1)
    
    def propose(self, state: State, inputs: tf.Tensor, observation: tf.Tensor):
        return self.sample(state, inputs)

## Observation Model

In [10]:
class VRNNNormalObservationModel(ObservationSampler):
    
    def __init__(self, latent_encoder, observation_size, name='VRNNObservationModel'):
        super(VRNNNormalObservationModel, self).__init__(name=name)
        # mlp parametrised gaussian
        self.emission = NNNormalDistribution(size=observation_size, 
                                             hidden_layer_sizes=[observation_size])
        
    
    def observation_dist(self, state: State):
        latent_state = state.particles
        latent_encoded = state.latent_encoded
        rnn_out = state.rnn_out
        rnn_out = tf.reshape(rnn_out, [state.batch_size, state.n_particles, rnn_hidden_size])
        dist = self.emission(latent_state, rnn_out) 
        return dist
    
    def loglikelihood(self, state: State, observation: tf.Tensor):
        
        dist = self.observation_dist(state)
        return tf.reduce_sum(dist.log_prob(observation), axis=-1)
        

    def sample(self, state: State):
        dist = self.observation_dist(state)
        
        
        return dist.sample()
        

In [11]:
class VRNNBernoulliObservationModel(ObservationSampler):
    
    def __init__(self, latent_encoder, observation_size, name='VRNNObservationModel'):
        super(VRNNNormalObservationModel, self).__init__(name=name)
        # mlp parametrised gaussian
        self.emission = NNNormalDistribution(size=observation_size, 
                                             hidden_layer_sizes=[observation_size])
        
    
    def observation_dist(self, state: State):
        latent_state = state.particles
        latent_encoded = state.latent_encoded
        rnn_out = state.rnn_out
        rnn_out = tf.reshape(rnn_out, [state.batch_size, state.n_particles, rnn_hidden_size])
        dist = self.emission(latent_state, rnn_out) 
        return dist
    
    def loglikelihood(self, state: State, observation: tf.Tensor):
        
        dist = self.observation_dist(state)
        return tf.reduce_sum(dist.log_prob(observation), axis=-1)
        

    def sample(self, state: State):
        dist = self.observation_dist(state)
        
        
        return dist.sample()
        

## Initial State

In [12]:
@attr.s
class VRNNState(State):
    ADDITIONAL_STATE_VARIABLES = ('rnn_state',) # rnn_out and encoded no need o be resampled
    rnn_state = attr.ib()
    rnn_out = attr.ib()
    latent_encoded = attr.ib()

In [13]:
transition_model = VRNNTransitionModel(rnn_hidden_size, data_encoder, latent_encoder)
observation_model = VRNNNormalObservationModel(latent_encoder, observation_size)
proposal_model = VRNNProposalModel(rnn_hidden_size, data_encoder, latent_encoder)

In [14]:
# initial state
normal_dist = tfp.distributions.Normal(0., 1.)
initial_latent_state = tf.zeros([batch_size, n_particles, dimension])
initial_latent_state = tf.cast(initial_latent_state, dtype=float)
latent_encoded = transition_model.latent_encoder(initial_latent_state)

# initial rnn_state
initial_rnn_state = [normal_dist.sample([batch_size,n_particles,rnn_hidden_size])]*2
initial_rnn_state = tf.concat(initial_rnn_state, axis=-1)

# rnn_out
initial_rnn_out = tf.zeros([batch_size * n_particles, rnn_hidden_size])

initial_weights = tf.ones((batch_size, n_particles), dtype=float) / tf.cast(n_particles, float)
log_likelihoods = tf.zeros(batch_size, dtype=float)
initial_state = VRNNState(particles=initial_latent_state, 
                          log_weights = tf.math.log(initial_weights),
                          weights=initial_weights, 
                          log_likelihoods=log_likelihoods,
                          rnn_state = initial_rnn_state,
                          rnn_out = initial_rnn_out,
                          latent_encoded = latent_encoded)


state = initial_state

## Check variables

In [17]:
# snt networks initiated on first call
t_samp = transition_model.sample(state, inputs[0])
obs_samp = observation_model.sample(state)

In [23]:
for var in transition_model.variables:
    print(var.name)

latent_encoder/linear_0/b:0
latent_encoder/linear_0/w:0
latent_encoder/linear_1/b:0
latent_encoder/linear_1/w:0
lstm_cell/kernel:0
lstm_cell/recurrent_kernel:0
lstm_cell/bias:0
conditional_normal_distribution_fcnet/linear_0/b:0
conditional_normal_distribution_fcnet/linear_0/w:0
conditional_normal_distribution_fcnet/linear_1/b:0
conditional_normal_distribution_fcnet/linear_1/w:0


In [22]:
for var in observation_model.variables:
    print(var.name)

conditional_normal_distribution_fcnet/linear_0/b:0
conditional_normal_distribution_fcnet/linear_0/w:0
conditional_normal_distribution_fcnet/linear_1/b:0
conditional_normal_distribution_fcnet/linear_1/w:0


## Particle Filter

In [18]:
# resampling
resampling_criterion = NeffCriterion(tf.constant(0.5), tf.constant(True))
systematic = SystematicResampler()
multinomial = MultinomialResampler()


epsilon = tf.constant(0.25)
scaling = tf.constant(0.9)

regularized = RegularisedTransform(epsilon, scaling=scaling, max_iter=1000, convergence_threshold=1e-4)

In [20]:
particle_filter = SMC(observation_model, transition_model, proposal_model, resampling_criterion, regularized)

recorded_states = particle_filter(initial_state, observation_series=obs_data, n_observations=T, inputs_series= inputs_data)

recorded_states  = attr.evolve(recorded_states)
